In [1]:
import sys
import biogeme.biogeme_logging as blog
import biogeme.exceptions as excep
import biogeme.biogeme as bio
import biogeme.distributions as dist
import biogeme.results as res
from biogeme import models
from biogeme.expressions import (
    Beta,
    RandomVariable,
    exp,
    log,
    Integrate,
)

from read_or_estimate import read_or_estimate

from optima import (
    read_data,
    male,
    age,
    haveChildren,
    highEducation,
       SocioProfCat,
    WaitingTimePT,
    Choice,
    TimePT_scaled,
    TimeCar_scaled,
    MarginalCostPT_scaled,
    CostCarCHF_scaled,
    distance_km_scaled,
    )

logger = blog.get_screen_logger(level=blog.INFO)
logger.info('Example m02_sequential_estimation_lin.py')

Example m02_sequential_estimation_lin.py 


In [3]:
MODELNAME = 'm01_latent_variable'
try:
    struct_results = res.bioResults(pickle_file=f'{MODELNAME}.pickle')
except excep.BiogemeError:
    print(
        f'Run first the script {MODELNAME}.py in order to generate the '
        f'file {MODELNAME}.pickle, and move it to the directory saved_results'
    )
    sys.exit()
struct_betas = struct_results.get_beta_values()

In [4]:
coef_intercept = struct_betas['coef_intercept']
coef_age_50_less = struct_betas['coef_age_50_less']
coef_male = struct_betas['coef_male']
coef_haveChildren = struct_betas['coef_haveChildren']
coef_highEducation = struct_betas['coef_highEducation']
coef_employees = struct_betas['coef_employees']


In [5]:
omega = RandomVariable('omega')
density = dist.normalpdf(omega)
sigma_s = Beta('sigma_s', 1, None, None, 0)

In [6]:
ACTIVELIFE = (
    coef_intercept
        + coef_highEducation * highEducation
       + coef_employees * (SocioProfCat == 6)
    + coef_age_50_less * (age <= 50)
    + coef_male * male
    + coef_haveChildren * haveChildren
    + sigma_s * omega
)

In [7]:
ASC_CAR = Beta('ASC_CAR', 0.94, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 1)
ASC_SM = Beta('ASC_SM', 0.35, None, None, 0)
BETA_DIST = Beta('BETA_DIST', -1.3, None, None, 0)
BETA_TIME_CAR_REF = Beta('BETA_TIME_CAR_REF', -6.1, None, 0, 0)
BETA_TIME_PT_REF = Beta('BETA_TIME_PT_REF', 0, None, 0, 0)
BETA_WAITING_TIME = Beta('BETA_WAITING_TIME', -0.075, None, None, 0)

In [8]:
BETA_TIME_PT_AL = Beta('BETA_TIME_PT_AL', 1.5, None, None, 0)
BETA_TIME_PT = BETA_TIME_PT_REF + BETA_TIME_PT_AL * ACTIVELIFE
BETA_TIME_CAR_AL = Beta('BETA_TIME_CAR_AL', -0.11, None, None, 0)
BETA_TIME_CAR = BETA_TIME_CAR_REF  + BETA_TIME_CAR_AL * ACTIVELIFE

In [9]:
V0 = (
    ASC_PT
    + BETA_TIME_PT * TimePT_scaled
    + BETA_WAITING_TIME * WaitingTimePT
)

V1 = (
    ASC_CAR
    + BETA_TIME_CAR * TimeCar_scaled

)

V2 = ASC_SM + BETA_DIST * distance_km_scaled

In [10]:
V = {0: V0, 1: V1, 2: V2}

In [11]:
condprob = models.logit(V, None, Choice)

In [12]:
loglike = log(Integrate(condprob * density, 'omega'))

In [13]:
database = read_data()

In [14]:
the_biogeme = bio.BIOGEME(database, loglike)
the_biogeme.modelName = 'm02_sequential_estimation_lin'
the_biogeme.maxiter = 1000

Biogeme parameters read from biogeme.toml. 


In [15]:
results = read_or_estimate(the_biogeme=the_biogeme, directory='saved_results')

As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
*** Initial values of the parameters are obtained from the file __m02_sequential_estimation_lin.iter 
Cannot read file __m02_sequential_estimation_lin.iter. Statement is ignored. 
As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Iter.         ASC_CAR          ASC_SM       BETA_DIST BETA_TIME_CAR_A BETA_TIME_CAR_R BETA_TIME_PT_AL BETA_TIME_PT_RE BETA_WAITING_TI         sigma_s     Function    Relgrad   Radius      Rho      
    0            0.94            0.35            -1.3           -0.11 

In [16]:
print(results.short_summary())

Results for model m02_sequential_estimation_lin
Nbr of parameters:		9
Sample size:			1906
Excluded data:			0
Final log likelihood:		-1236.934
Akaike Information Criterion:	2491.868
Bayesian Information Criterion:	2541.842



In [17]:
print(f'Final log likelihood: {results.data.logLike:.3f}')
print(f'Output file: {results.data.htmlFileName}')

Final log likelihood: -1236.934
Output file: m02_sequential_estimation_lin.html


In [18]:
results.get_estimated_parameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.682018,0.123681,5.514325,3.501212e-08
ASC_SM,0.451723,0.171385,2.635723,8.395834e-03
BETA_DIST,-2.068373,0.050665,-40.824369,0.000000e+00
BETA_TIME_CAR_AL,-1.556741,0.215735,-7.215971,5.355716e-13
BETA_TIME_CAR_REF,-17.799599,1.046750,-17.004639,0.000000e+00
BETA_TIME_PT_AL,-0.496608,0.063052,-7.876104,3.330669e-15
BETA_TIME_PT_REF,-6.559461,0.520044,-12.613292,0.000000e+00
BETA_WAITING_TIME,-0.028667,0.008144,-3.520106,4.313737e-04
sigma_s,10.143876,1.320111,7.684109,1.532108e-14
